## Классификация текстов
Цель работы: изучить методы классификации текстов на основе различных способов векторизации.

Задание: Реализовать два подхода к классификации текстов:

- На основе TfidfVectorizer.
- На основе word2vec, обученного вручную на корпусе.
- Сравнить полученные результаты классификации.

In [4]:
# !pip install numpy==1.24.3
# !pip install gensim

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import nltk
# import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

IMDB dataset having 50K movie reviews for natural language processing or Text analytics.

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms.

In [3]:
file_path = 'IMDB_Dataset.csv'
df = pd.read_csv(file_path, low_memory=False)
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

## Способ 1. Классификация на основе TfidfVectorizer
Для первого подхода тексты были преобразованы в числовые векторы с помощью TfidfVectorizer, исключающего стоп-слова и учитывающего частотность слов.

Классификация выполнялась с помощью модели LogisticRegression, которая хорошо работает на задачах с разреженными признаками.

In [10]:
# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Векторизация текста
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_test_vec)
target_names = [str(cls) for cls in np.unique(y_test)]
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



## Способ 2. Классификация на основе Word2Vec

Во втором подходе используется модель Word2Vec, обученная на тексте из датасета 20 Newsgroups.
Каждое слово кодируется вектором, отражающим его контекст, а вектор документа получаем как среднее всех слововых векторов.

Обучение выполняется с использованием библиотеки gensim.

In [6]:
nltk.download('punkt')
# nltk.download('punkt_tab')  # иногда помогает отдельно
# nltk.download('popular')    # качает всё базовое, включая punkt и токенизаторы

[nltk_data] Downloading package punkt to /Users/artisia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/artisia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/artisia/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/artisia/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/artisia/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/artisia/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/artisia

True

In [11]:
# Токенизируем все тексты для обучения модели
tokenized_texts = [word_tokenize(text.lower()) for text in df['review']]

# Обучаем Word2Vec на корпусе
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=2, workers=4, sg=1)

# Функция усреднения эмбеддингов слов в документе
def document_vector(doc, model):
    words = [word for word in word_tokenize(doc.lower()) if word in model.wv.key_to_index]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

# Преобразуем все документы в векторы
X_w2v = np.array([document_vector(text, w2v_model) for text in df['review']])
y = df['sentiment']

# Делим выборку
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, random_state=42)

После получения векторного представления документов, была обучена модель LogisticRegression.
Модель показала определённый уровень качества классификации, позволяющий сравнить её с результатами, полученными при использовании TfidfVectorizer.

In [13]:
# Обучаем модель на векторах Word2Vec
model_w2v = LogisticRegression(max_iter=1000)
model_w2v.fit(X_train_w2v, y_train_w2v)

# Предсказания
y_pred_w2v = model_w2v.predict(X_test_w2v)

# Выводим отчёт
print(classification_report(y_test_w2v, y_pred_w2v, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.87      0.86      0.86      4961
    positive       0.86      0.87      0.87      5039

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000



## Сравнение моделей и вывод

В данной лабораторной работе были реализованы два подхода к классификации текстов:

TfidfVectorizer + LogisticRegression
Модель показала высокую точность благодаря учёту частоты слов в тексте и их значимости в корпусе.
Преимущество — простота и эффективность на небольших и средних наборах данных.

Word2Vec + LogisticRegression
Этот подход использует контекстные представления слов, усреднённые по каждому документу.
Хотя результат может быть ниже, модель способна улавливать семантику слов и подходит для более глубокого анализа текста.

Обе модели успешно решили задачу классификации. Метод на основе TfidfVectorizer показал более высокую точность на выбранном корпусе, однако Word2Vec даёт более гибкое представление текста и может быть полезен при работе с более сложными задачами или большими данными.